In [ ]:
!pip install PyMuPDF
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.9/474.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 

# Step 2: Converting PDFs to Markdown Using DeepSeek-OCR

This notebook demonstrates the second step in building a RAG (Retrieval-Augmented Generation) pipeline: **converting research papers from PDF format to structured Markdown**.

## Why Convert PDFs to Markdown?

After collecting research papers in Step 1, we need to extract their content in a format that's:
- **Readable by LLMs**: Clean text without PDF artifacts
- **Structured**: Preserves headings, sections, lists, and tables
- **Searchable**: Easy to index and retrieve
- **Math-friendly**: Handles LaTeX equations properly

## What is DeepSeek-OCR?

[DeepSeek-OCR](https://huggingface.co/deepseek-ai/DeepSeek-OCR) is a state-of-the-art vision-language model specifically designed for:
- Document understanding and OCR
- Converting images to text
- Preserving document structure
- Handling mathematical formulas and scientific notation
- Supporting multiple output formats (text, JSON, markdown)

## Overview of This Step

In this notebook, we will:
1. Load the DeepSeek-OCR model using vLLM
2. Convert PDF pages to images
3. Process each page through the OCR model
4. Combine pages into a single Markdown document
5. Save structured markdown files for each paper

## Why This Matters for RAG

Converting PDFs to markdown is critical for:
- **Better chunking**: Split documents into meaningful sections
- **Improved embeddings**: Clean text produces better vector representations
- **Enhanced retrieval**: Structured content is easier to search


## PDF files

In [ ]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = Path("/content/drive/MyDrive/RAG")
PDF_FOLDER = BASE_PATH / "block1_output_pdfs"

if PDF_FOLDER.exists():
    print("Ready for Block2!")
else:
    print(f"{PDF_FOLDER} does not exist")


for file in PDF_FOLDER.iterdir():
    print(file.name)


OUTPUT_TYPE = "markdown"
OUTPUT_PATH = PDF_FOLDER.parent / "block2_output_markdowns"

# Create output directory if it doesn't exist
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

print(f"   PDF Folder: {PDF_FOLDER}")
print(f"   Output Type: {OUTPUT_TYPE}")
print(f"   Markdown Path: {OUTPUT_PATH}")


Mounted at /content/drive
Ready for Block2!
14296.pdf
14294.pdf
14293.pdf
14290.pdf
14287.pdf
14286.pdf
14284.pdf
14282.pdf
14280.pdf
14279.pdf
   PDF Folder: /content/drive/MyDrive/RAG/block1_output_pdfs
   Output Type: markdown
   Markdown Path: /content/drive/MyDrive/RAG/block2_output_markdowns


## Import Required Libraries

In [ ]:
import os
import json
from pathlib import Path
from typing import List
import fitz  # PyMuPDF
from PIL import Image
from vllm import LLM, SamplingParams
from vllm.model_executor.models.deepseek_ocr import NGramPerReqLogitsProcessor

print("Libraries imported successfully")

Libraries imported successfully


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/image_processing_auto.py:647: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


## Define PDF to Images Conversion Function

The first step in OCR is to convert each PDF page to an image. This function:
- Creates a directory for images (e.g., `14255_images/`)
- Converts each PDF page to a high-resolution PNG image
- Returns a list of image paths for processing

In [ ]:
def pdf_to_images(pdf_path: Path) -> List[str]:
    """
    Convert PDF pages to high-resolution images.

    Args:
        pdf_path: Path to the PDF file

    Returns:
        List of paths to generated images
    """
    output_dir = pdf_path.parent / f"{pdf_path.stem}_images"
    output_dir.mkdir(exist_ok=True)

    doc = fitz.open(pdf_path)
    image_paths = []

    for page_num in range(len(doc)):
        page = doc[page_num]
        # Use 2x scaling for high resolution
        pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
        image_path = output_dir / f"page_{page_num + 1}.png"
        pix.save(image_path)
        image_paths.append(str(image_path.absolute()))

    doc.close()
    return image_paths

print("✓ PDF to images function defined")

✓ PDF to images function defined


## Define PDF Processing Function

This is the core function that:
1. Converts PDF to images
2. Prepares prompts for the OCR model (markdown mode with grounding)
3. Runs the model on each page
4. Combines results from all pages

### Prompt Strategy

- **Markdown mode**: Uses `<|grounding|>` mode to properly extract formulas and document structure
- **Grounding**: Helps the model understand document layout and preserve formatting

In [ ]:
def process_pdf(llm, pdf_path: Path, sampling_params, out_type: str = "markdown") -> str:
    """
    Process a single PDF and extract text using DeepSeek-OCR.

    Args:
        llm: Initialized vLLM model instance
        pdf_path: Path to the PDF file
        sampling_params: Sampling parameters for generation
        out_type: Output format ('markdown', 'txt', or 'json')

    Returns:
        Extracted and formatted text from all pages
    """
    print(f"Processing: {pdf_path.name}")

    # Convert to images
    image_paths = pdf_to_images(pdf_path)
    print(f"  - Converted {len(image_paths)} pages to images")

    # Prepare inputs with correct prompt format
    # For markdown: Use grounding mode to properly extract formulas and structure
    if out_type == "markdown":
        prompt = "<image>\n<|grounding|>Convert the document to markdown."
    else:
        prompt = "<image>\nFree OCR."

    inputs = []
    for image_path in image_paths:
        image = Image.open(image_path).convert("RGB")
        inputs.append({
            "prompt": prompt,
            "multi_modal_data": {"image": image}
        })

    # Run OCR
    print(f"  - Running OCR on {len(image_paths)} pages...")
    outputs = llm.generate(inputs, sampling_params)

    # Combine pages
    full_text = ""
    for i, output in enumerate(outputs, 1):
        page_text = output.outputs[0].text
        full_text += f"\n\n--- Page {i} ---\n\n{page_text}"

    return full_text.strip()

print("✓ PDF processing function defined")

✓ PDF processing function defined


## Define Output Saving Function

This function saves the extracted text in the specified format:
- **Markdown (.md)**: Structured format with preserved formatting
- **Text (.txt)**: Plain text output
- **JSON (.json)**: Structured data with metadata

In [ ]:
def save_output(text: str, pdf_path: Path, out_path: Path, out_type: str):
    """
    Save extracted text to output file in the specified format.

    Args:
        text: Extracted text content
        pdf_path: Original PDF path (used for naming)
        out_path: Output directory path
        out_type: Output format ('txt', 'json', or 'markdown')

    Returns:
        Path to the saved output file
    """
    if out_type == "txt":
        output_file = out_path / f"{pdf_path.stem}.txt"
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(text)
    elif out_type == "json":
        output_file = out_path / f"{pdf_path.stem}.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump({"file": pdf_path.name, "text": text}, f, indent=2, ensure_ascii=False)
    elif out_type == "markdown":
        output_file = out_path / f"{pdf_path.stem}.md"
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(text)
    else:
        raise ValueError(f"Unsupported output type: {out_type}")

    print(f"  - Saved to: {output_file}")
    return output_file

print("✓ Output saving function defined")

✓ Output saving function defined


## Configure Processing Parameters

Set up the parameters for our OCR processing pipeline:
- **Input folder**: Directory containing PDFs from Step 1
- **Output format**: Markdown for structured content
- **Output path**: Directory to save markdown files

In [ ]:


# Get list of PDF files
pdf_files = list(PDF_FOLDER.glob("*.pdf"))
print(f"\nFound {len(pdf_files)} PDF file(s) to process")

if pdf_files:
    print("\nPDFs to process:")
    for i, pdf in enumerate(pdf_files[:5], 1):
        print(f"   {i}. {pdf.name}")
    if len(pdf_files) > 5:
        print(f"   ... and {len(pdf_files) - 5} more")


📁 Found 10 PDF file(s) to process

📚 PDFs to process:
   1. 14296.pdf
   2. 14294.pdf
   3. 14293.pdf
   4. 14290.pdf
   5. 14287.pdf
   ... and 5 more


## Initialize the DeepSeek-OCR Model

This is a crucial step that loads the model into memory. The model initialization includes:

### Key Parameters

1. **enable_prefix_caching=False**: Disabled as recommended by DeepSeek documentation
2. **mm_processor_cache_gb=0**: Reduces memory usage for multi-modal processing
3. **logits_processors**: Uses NGram processor for better output quality
4. **trust_remote_code=True**: Required for custom model architecture

### Sampling Parameters

- **temperature=0.0**: Deterministic output for consistent results
- **max_tokens=8192**: Allows for long documents
- **ngram_size=30, window_size=90**: Controls repetition prevention
- **whitelist_token_ids**: Allows specific tokens for HTML table elements

**Note**: This may take a few minutes to load the model into GPU memory.

In [ ]:
print("Initializing DeepSeek-OCR with vLLM...")
print("This may take a few minutes to load the model into GPU memory...")
print()

# Get the local model path
model_id = "deepseek-ai/DeepSeek-OCR"

# Initialize the model following DeepSeek-OCR best practices
llm = LLM(
    model=model_id,
    enable_prefix_caching=False,  # As per documentation
    mm_processor_cache_gb=0,      # As per documentation
    logits_processors=[NGramPerReqLogitsProcessor],  # Required for quality output
    trust_remote_code=True,
)



🔧 Initializing DeepSeek-OCR with vLLM...
   This may take a few minutes to load the model into GPU memory...

INFO 01-17 02:12:22 [utils.py:253] non-default args: {'trust_remote_code': True, 'enable_prefix_caching': False, 'disable_log_stats': True, 'mm_processor_cache_gb': 0, 'logits_processors': [<class 'vllm.model_executor.models.deepseek_ocr.NGramPerReqLogitsProcessor'>], 'model': 'deepseek-ai/DeepSeek-OCR'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

INFO 01-17 02:12:44 [model.py:514] Resolved architecture: DeepseekOCRForCausalLM
WARNING 01-17 02:12:44 [model.py:1955] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 01-17 02:12:44 [model.py:2005] Casting torch.bfloat16 to torch.float16.
INFO 01-17 02:12:44 [model.py:1661] Using max model len 8192
INFO 01-17 02:12:48 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

WARNING 01-17 02:12:50 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 01-17 02:20:02 [llm.py:360] Supported tasks: ['generate']


In [ ]:
# Configure sampling parameters
sampling_params = SamplingParams(
    temperature=0.0,              # Deterministic output
    max_tokens=8192,              # Increased for long documents
    # NGram logit processor args for better output quality
    extra_args=dict(
        ngram_size=30,
        window_size=90,
        whitelist_token_ids={128821, 128822},  # whitelist: <td>, </td>
    ),
    skip_special_tokens=False,    # As per documentation
)

print("\nModel loaded successfully!")
print("="*80)


✅ Model loaded successfully!


## Process All PDFs

Now we'll process each PDF file:
1. Convert PDF pages to images
2. Run OCR on each page
3. Combine results
4. Save as markdown

**Note**: This can take several minutes depending on:
- Number of PDFs
- Number of pages per PDF
- GPU availability and speed

In [ ]:
print("\nStarting PDF processing...")
print("="*80)
print()

successful_files = []
failed_files = []

# Process each PDF
for i, pdf_path in enumerate(pdf_files, 1):
    try:
        print(f"[{i}/{len(pdf_files)}] {pdf_path.name}")

        # Process the PDF
        text = process_pdf(llm, pdf_path, sampling_params, OUTPUT_TYPE)

        # Save the output
        output_file = save_output(text, pdf_path, OUTPUT_PATH, OUTPUT_TYPE)
        successful_files.append(pdf_path.name)

        print(f"Successfully processed {pdf_path.name}")
        print()

    except Exception as e:
        print(f"ERROR: {e}")
        failed_files.append(pdf_path.name)
        print()
        continue

print("="*80)
print("\n  Processing Summary:")
print(f"   Total PDFs: {len(pdf_files)}")
print(f"   Successful: {len(successful_files)}")
print(f"   Failed: {len(failed_files)}")

if failed_files:
    print(f"\nFailed files:")
    for fname in failed_files:
        print(f"   - {fname}")


🔄 Starting PDF processing...

[1/10] 14296.pdf
Processing: 14296.pdf
  - Converted 10 pages to images
  - Running OCR on 10 pages...


Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  - Saved to: /content/drive/MyDrive/RAG/block2_output_markdowns/14296.md
  ✅ Successfully processed 14296.pdf

[2/10] 14294.pdf
Processing: 14294.pdf
  - Converted 11 pages to images
  - Running OCR on 11 pages...


Adding requests:   0%|          | 0/11 [00:00<?, ?it/s]

...

Processed prompts:   0%|          | 0/38 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  - Saved to: /content/drive/MyDrive/RAG/block2_output_markdowns/14280.md
  ✅ Successfully processed 14280.pdf

[10/10] 14279.pdf
Processing: 14279.pdf
  - Converted 15 pages to images
  - Running OCR on 15 pages...


Adding requests:   0%|          | 0/15 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/15 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  - Saved to: /content/drive/MyDrive/RAG/block2_output_markdowns/14279.md
  ✅ Successfully processed 14279.pdf


📊 Processing Summary:
   Total PDFs: 10
   Successful: 10
   Failed: 0


## Verify Output Files

Let's check what files were created in the output directory:

In [ ]:
# List output files
output_files = list(OUTPUT_PATH.glob(f"*.{OUTPUT_TYPE.replace('markdown', 'md')}"))

print(f"Output Directory: {OUTPUT_PATH}")
print(f"Generated {len(output_files)} markdown files\n")

if output_files:
    print("Output files:")
    for i, file in enumerate(sorted(output_files)[:10], 1):
        file_size = file.stat().st_size / 1024  # KB
        print(f"   {i:2d}. {file.name} ({file_size:.1f} KB)")

    if len(output_files) > 10:
        print(f"   ... and {len(output_files) - 10} more files")

    # Calculate total size
    total_size = sum(f.stat().st_size for f in output_files) / (1024 * 1024)  # MB
    print(f"\nTotal size: {total_size:.2f} MB")

📁 Output Directory: /content/drive/MyDrive/RAG/block2_output_markdowns
📄 Generated 10 markdown files

📝 Output files:
    1. 14279.md (64.3 KB)
    2. 14280.md (155.1 KB)
    3. 14282.md (73.6 KB)
    4. 14284.md (93.3 KB)
    5. 14286.md (130.2 KB)
    6. 14287.md (226.7 KB)
    7. 14290.md (81.2 KB)
    8. 14293.md (51.8 KB)
    9. 14294.md (41.0 KB)
   10. 14296.md (42.9 KB)

📊 Total size: 0.94 MB


## Preview Sample Output

Let's look at a sample of the extracted markdown to verify quality:

In [ ]:
# Read and display first few lines from a sample file
if output_files:
    sample_file = output_files[0]
    print(f"📄 Sample from: {sample_file.name}")
    print("="*80)

    with open(sample_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Show first 1000 characters
    preview_length = 1000
    print(content[:preview_length])

    if len(content) > preview_length:
        print(f"\n... (showing first {preview_length} characters of {len(content)} total)")
else:
    print("No output files found to preview.")

📄 Sample from: 14296.md
--- Page 1 ---

<|ref|>title<|/ref|><|det|>[[174, 98, 823, 172]]<|/det|>
# NEUROACOUSTIC PATTERNS: CONSTANT Q CEP- STRAL COEFFICIENTS FOR THE CLASSIFICATION OF NEURODEGENERATIVE DISORDERS  

<|ref|>text<|/ref|><|det|>[[183, 196, 398, 224]]<|/det|>
Anonymous authors Paper under double- blind review  

<|ref|>sub_title<|/ref|><|det|>[[452, 260, 544, 275]]<|/det|>
## ABSTRACT  

<|ref|>text<|/ref|><|det|>[[232, 291, 764, 568]]<|/det|>
Early identification of neurodegenerative diseases is crucial for effective diagnosis in neurological disorders. However, the quasi- periodic nature of vocal tract sampling often results in inadequate spectral resolution in traditional spectral features, such as Mel Frequency Cepstral Coefficients (MFCC), thereby limiting their classification effectiveness. In this study, we propose the use of Constant Q Cepstral Coefficients (CQCC), which leverage geometrically spaced frequency bins to provide superior spectrotemporal resolution, par

## Summary of What We've Accomplished

In this notebook, we successfully:

1. ✅ **Loaded DeepSeek-OCR Model**: Initialized the state-of-the-art OCR model with optimal parameters
2. ✅ **Converted PDFs to Images**: Extracted high-resolution images from each PDF page
3. ✅ **Processed Documents**: Used the OCR model to extract structured content
4. ✅ **Generated Markdown Files**: Created clean, structured markdown from academic papers
5. ✅ **Preserved Formatting**: Maintained document structure including headings, lists, and formulas

### Data Pipeline Progress

```
Step 1: Fetch Papers from OpenReview
         ↓
   [PDF Files in block1_output_pdfs/]
         ↓
Step 2: Convert PDFs to Markdown (THIS STEP) ✅
         ↓
   [Markdown Files in block2_output_markdowns/]
         ↓
Step 3: Chunk and Embed (NEXT STEP)
         ↓
   [Vector Database]
         ↓
Step 4: Build RAG Query System
         ↓
   [Question Answering Interface]
```

### Files Created

- **`block2_output_markdowns/*.md`**: Markdown files for each paper
- **`block1_output_pdfs/*_images/`**: Intermediate image files (can be deleted if needed)

### Quality Indicators

The markdown output should include:
- ✓ Proper heading hierarchy
- ✓ Mathematical formulas (LaTeX notation)
- ✓ Tables and lists
- ✓ References and citations
- ✓ Section structure

## Next Step: Text Chunking and Embedding

### What's Next in the RAG Pipeline?

Now that we have clean, structured markdown files, the next step is to **prepare the text for retrieval**. This involves:

#### 1. Text Chunking

Breaking documents into smaller, meaningful pieces:
- **Why?** Large documents need to be split for efficient retrieval
- **How?** Use semantic chunking that respects section boundaries
- **Goal:** Each chunk should contain a coherent piece of information

#### 2. Generating Embeddings

Convert text chunks into vector representations:
- **Why?** Enable semantic similarity search
- **Models:** Sentence transformers, OpenAI embeddings, etc.
- **Output:** Dense vector representations (e.g., 768 or 1536 dimensions)

#### 3. Vector Database Storage

Store embeddings for fast retrieval:
- **Options:** FAISS, Pinecone, Weaviate, Chroma, etc.
- **Features:** Approximate nearest neighbor search
- **Performance:** Sub-millisecond retrieval times

#### 4. Building the RAG System

Combine retrieval with generation:
- **Query Processing:** Convert questions to embeddings
- **Retrieval:** Find most relevant chunks
- **Generation:** Use LLM to answer based on retrieved context
- **Citation:** Link answers back to source papers

### Key Advantages of Our Approach

By using structured markdown:
1. **Better Chunking**: Can split on headings and sections
2. **Context Preservation**: Maintain document structure in chunks
3. **Improved Retrieval**: Structured text yields better embeddings
4. **Enhanced Citations**: Easy to reference specific sections

### Typical RAG Pipeline Metrics

A well-built RAG system should achieve:
- **Retrieval Precision**: 70-90% of top-k results are relevant
- **Answer Quality**: High factual accuracy with proper citations
- **Response Time**: < 2 seconds for query to answer
- **Scalability**: Handle thousands to millions of documents

---

** Ready for Step 3?** The next notebook will cover:
- Chunking strategies for academic papers
- Embedding generation
- Vector database setup
